## Συγκέντρωση των τίτλων

Αρχικά με τις βιβλιοθήκες `requests` και `BeautifulSoup` συγκέντρωσα όλους τους τίτλους των ταινιών από την ιστοσελίδα του Film Pit και τις αποθήκευσα σε ένα αρχείο με όνομα `movie_titles.csv`.

In [1]:
# Φόρτωση των απαραίτητων βιβλιοθηκών
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
from os import path

In [2]:
# Ελέγχουμε έαν υπάρχει ήδη το αρχείο αλλιώς το δημιουργούμε
if path.exists(path.join('datasets','movie_titles.csv')):
    movie_titles = pd.read_csv(path.join('datasets', 'movie_titles.csv')).squeeze('columns')
else:
    URL = "https://thefilmpit.com"

    def get_movies_title(URL):
        """Scrapes podcasts hrefs to get movie titles"""
        movie_titles = []
        page = requests.get(URL)
        next_link = None   
        
        if page.ok:
            soup = BeautifulSoup(page.content, 'html.parser')
            try:
                # Ελέγχουμε αν υπάρχει επόμενη σελίδα στο site
                next_link = soup.find('link', {'rel':'next',}).get('href')
            except AttributeError:
                print("This page hasn't a next link")

            # Απο τους συνδέσμους href λαμβάνουμε τους τίτλους
            for podcast in soup.find_all('a', attrs={'rel':'bookmark'}): 
                podcast_link = podcast.get('href')
                title = podcast_link.split('/')[-2].replace('-', ' ')
                movie_titles.append(title)
        return movie_titles, next_link

    next_link = URL
    movie_titles = []

    while next_link:
        print(f"Scraping next link: {next_link}")
        titles, next_link = get_movies_title(next_link)
        movie_titles += titles
        time.sleep(5)
    print("Scraping finished.")

Scraping next link: https://thefilmpit.com
Scraping next link: https://thefilmpit.com/page/2/
Scraping next link: https://thefilmpit.com/page/3/
Scraping next link: https://thefilmpit.com/page/4/
Scraping next link: https://thefilmpit.com/page/5/
Scraping next link: https://thefilmpit.com/page/6/
Scraping next link: https://thefilmpit.com/page/7/
Scraping next link: https://thefilmpit.com/page/8/
Scraping next link: https://thefilmpit.com/page/9/
Scraping next link: https://thefilmpit.com/page/10/
Scraping next link: https://thefilmpit.com/page/11/
Scraping next link: https://thefilmpit.com/page/12/
This page hasn't a next link
Scraping finished.


Κάποιοι τίτλοι χρειάζονταν διόρθωση όπως βλέπετε παρακάτω.

In [3]:
if not path.exists(path.join('datasets','movie_titles.csv')):
    movie_titles += ['never too young to die', 'Dr Caligari', 'Yeti'] # 3 ταινίες δεν υπάρχουν στην ιστοσελίδα
    movie_titles = pd.Series(movie_titles)
    
    fixed_titles = {'diagalaxiaki poiotita galaxy of terror': 'galaxy of terror',
                    'i scholi tou gkontfrei sakura killers': 'sakura killers',
                    'exairetika petsino podkast the punisher': 'the punisher',
                    'o rambu tis indonisias einai edo': 'Rambu aka The Intruder',
                    'tha einai san star gouorz alla den tha einai star gouorz battle beyond the stars': 'battle beyond the stars',
                    'brady idrotas kai pioti tie night stalker': 'night stalker',
                    'pao na kano penintarika double dragon': 'double dragon',
                    'mousikorama shock em dead': 'shock em dead',
                    'asiatiki tourne 3 undefeatable': 'undefeatable',
                    'asiatiki tourne 2 w is war': 'w is war',
                    'asiatiki tourne 1 for your height only': 'for your height only',
                    'ena mikro mousiko breik rappin': 'rappin',
                    'dark night of the scarecrow feat elina dimitriadi': 'dark night of the scarecrow',
                    'prom night feat elina dimitriadi': 'prom night',
                    'aerobicide': 'Killer Workout',
                    'an american hippie in paris': 'An American Hippie in Israel',
                    'tc2000': 'tc 2000',
                    'american ninia': 'american ninja',
                    'class of nukem high': "Class of Nuke 'Em High",
                    'superman 4': "superman iv"
                    }

    movie_titles.replace(fixed_titles, inplace=True)
    movie_titles.drop([91, 109], inplace=True)

In [4]:
# Αποθήκευση των τίτλων στο αρχείο movie_title.csv

if not path.exists(path.join('datasets','movie_titles.csv')):
    movie_titles.to_csv(path.join('datasets','movie_titles.csv'), index=False, header=['titles'])

## Εξαγωγή πληροφοριών από την ιστοσελίδα The Movie Database

Για την εξαγωγή πληροφοριών από το [TMDB](https://www.themoviedb.org) χρησιμοποίησα την βιβλιοθήκη `tmdbv3api`.

In [5]:
if path.exists(path.join('datasets','movies_tmdb.csv')):
    movies_tmdb = pd.read_csv(path.join('datasets','movies_tmdb.csv'), index_col=0)
else:
    from tmdbv3api import TMDb
    from config import config
    tmdb = TMDb()
    tmdb.api_key = config['tmdb_api_key'] # το api key δίνεται δωρεάν με απλή εγγραφή στο TMDB

    from tmdbv3api import Movie
    movie = Movie()

    basic_info = {}
    not_found = []
    for title in movie_titles.to_list():
        print('Fetching ' + title)
        try:
            # Χρησιμοποιούμε το πρώτο αποτέλεσμα αναζήτησης
            basic_info[title] = movie.search(title)[0] 
        except IndexError:
            # Αποθηκεύουμε τους τίτλους που δεν βρέθηκαν στην λίστα not_found
            not_found.append(title) 

    print('Movies not found: ', not_found)

Fetching broken america the killing of america
Fetching broken america star time
Fetching edge of honour
Fetching mind trap aka danger usa
Fetching caged fury
Fetching silk
Fetching raw nerve
Fetching raw justice
Fetching the baby
Fetching dont go in the house
Fetching conquest
Fetching battletruck
Fetching ninja squad
Fetching ultimax force
Fetching punk vacation
Fetching suburbia
Fetching An American Hippie in Israel
Fetching blood beach
Fetching hunters of the golden cobra
Fetching intent to kill
Fetching maniac cop
Fetching hard ticket to hawaii
Fetching warlock
Fetching maximum overdrive
Fetching nekromantik
Fetching manos
Fetching superman iv
Fetching flash gordon
Fetching surf nazis must die
Fetching to kako
Fetching zardoz
Fetching dark angel
Fetching girl in room 2a
Fetching tc 2000
Fetching interzone
Fetching zaat
Fetching american ninja
Fetching krull
Fetching beastmaster
Fetching raiders of the living dead
Fetching 315
Fetching q winged serpent
Fetching karate warrior 6
Fet

In [6]:
# Ελέγχουμε ότι το πρώτο αποτέλεσμα αναζήτησης ήταν το σωστό
if not path.exists(path.join('datasets','movies_tmdb.csv')):
    for title in movie_titles.to_list():
        if basic_info.get(title):
            print(title, ': ', basic_info[title]['title'], basic_info[title]['release_date'][:4])

caged fury :  Caged Fury 1990
silk :  Silk Road 2021
raw nerve :  Raw Nerve 1991
raw justice :  Raw Justice 1994
the baby :  The Boss Baby: Family Business 2021
dont go in the house :  Don't Go in the House 1979
conquest :  Game of Thrones - Conquest & Rebellion: An Animated History of the Seven Kingdoms 2017
battletruck :  Warlords of the 21st Century 1982
ninja squad :  The Ninja Squad 1986
ultimax force :  Ultimax Force 1987
punk vacation :  Punk Vacation 1990
suburbia :  Infidelity in Suburbia 2017
An American Hippie in Israel :  An American Hippie in Israel 1972
blood beach :  Blood Beach 1980
hunters of the golden cobra :  The Hunters of the Golden Cobra 1982
intent to kill :  Intent to Kill 1992
maniac cop :  Maniac Cop 1988
hard ticket to hawaii :  Hard Ticket to Hawaii 1987
warlock :  Warlock: The Armageddon 1993
maximum overdrive :  Maximum Overdrive 1986
nekromantik :  Nekromantik 1987
manos :  Manos: The Hands of Fate 1966
superman iv :  Superman IV: The Quest for Peace 198

Δυστυχώς τα πρώτο αποτέλεσμα από κάθε ταινία δεν είναι πάντα το επιθυμητό δεδομένου ότι πολλές ταινίες έχουν τον ίδιο ή παρόμοιο τίτλο. Παρακάτω επιλέγουμε τις σωστές ταινίες μέσω του αριθμού `id` κάθε ταινίας και προσθέτουμε και όσες δεν βρέθηκαν στην αναζήτηση.

In [7]:
if not path.exists(path.join('datasets','movies_tmdb.csv')):
    
    
    wrong_matches = {'Rambu aka The Intruder': '81944',
                     'lambada set the night on fire': '117269',
                     'the baby': '28156',
                     'conquest': '27232',
                     'suburbia': '28054',
                     'warlock': '11342',
                     'to kako': '39897',
                     'commander': '205697',
                     'captain america': '13995',
                     'star wars holiday special': '74849',
                     'the punisher': '8867',
                     'night stalker':'66474',
                     'the perfect weapon': '34421',
                     'the rage': '114936',
                     'warbirds': '219359',
                     'arena': '44796',
                     'jack frost': '27318',
                     'elves': '30452',
                     'prom night': '36599',
                     'cheerleader camp': '40087',
                     'endgame': '28850',
                     'thunder': '109104',
                     'Dr Caligari': '35642',
                     'Yeti': '92316'}
    

    movie_ids = {title:info['id'] for title, info in basic_info.items()}
    movie_ids.update(wrong_matches)

Εφόσον πλέον έχουμε το σωστό `id` για όλες τις ταινίες εξάγουμε όλες τις πληροφορίες που χρειαζόμαστε, τις αποθηκεύουμε στο λεξικό `movie_records` και με την βιβλιοθήκη `pandas` δημιουργούμε το αρχείο `movies_tmdb.csv`.

In [8]:
if not path.exists(path.join('datasets','movies_tmdb.csv')):
    print('Starting api request')
    movie_records = {}
    for title, id in movie_ids.items():
        
        print(f"Fetching {title}")
        mov = movie.details(id)
        temp_list = [
            mov.imdb_id,
            mov.original_title,
            mov.budget,
            mov.revenue,
            mov.runtime,
            mov.popularity,
            [company['name'] for company in mov.production_companies],
            [key['name'] for key in mov.keywords.keywords],
            [act['name'] for act in mov.casts.cast],
            mov.overview
        ]
        movie_records[mov.title] = temp_list
    
    print('TMDB request finished')
    columns = ['imdb_id',
               'original_title',
               'budget',
               'revenue',
               'runtime',
               'popularity',
               'production_companies',
               'keywords',
               'cast',
               'overview'
              ]
    movies_tmdb = pd.DataFrame.from_dict(movie_records, orient='index', columns=columns)

Starting api request
Fetching caged fury
Fetching silk
Fetching raw nerve
Fetching raw justice
Fetching the baby
Fetching dont go in the house
Fetching conquest
Fetching battletruck
Fetching ninja squad
Fetching ultimax force
Fetching punk vacation
Fetching suburbia
Fetching An American Hippie in Israel
Fetching blood beach
Fetching hunters of the golden cobra
Fetching intent to kill
Fetching maniac cop
Fetching hard ticket to hawaii
Fetching warlock
Fetching maximum overdrive
Fetching nekromantik
Fetching manos
Fetching superman iv
Fetching flash gordon
Fetching surf nazis must die
Fetching to kako
Fetching zardoz
Fetching dark angel
Fetching girl in room 2a
Fetching tc 2000
Fetching interzone
Fetching zaat
Fetching american ninja
Fetching krull
Fetching beastmaster
Fetching raiders of the living dead
Fetching 315
Fetching q winged serpent
Fetching karate warrior 6
Fetching savage streets
Fetching amsterdamned
Fetching fatal deviation
Fetching black gestapo
Fetching ss experiment love

In [9]:
# Αντικαθιστούμε το 0 ως τιμή που λείπει
movies_tmdb[['budget', 'revenue', 'runtime']] = movies_tmdb[['budget', 'revenue', 'runtime']].replace({0: np.nan})

Παρακάτω είναι ένα μικρό δείγμα του αρχείου.

In [10]:
movies_tmdb.sample(5)

,imdb_id,original_title,budget,revenue,runtime,popularity,production_companies,keywords,cast,overview
Blood Beach,tt0082083,Blood Beach,2000000.0,NaN,92.0,8.242,"[Compass International Pictures, Empress Film ...","[beach, monster, sand, spoof]","[David Huffman, Marianna Hill, Burt Young, Joh...",Something or someone is attacking people one b...
Elves,tt0099496,Elves,NaN,NaN,89.0,4.980,"[Triangle Film Corporation, Fitzgerald Film Co...","[elves, cat, experiment, nazi, family secrets,...","[Dan Haggerty, Julie Austin, Deanna Lund, Bora...",A young woman discovers that she is the focus ...
Killer Workout,tt0091339,Killer Workout,NaN,NaN,84.0,5.634,[Shapiro Entertainment],"[shower, stalker, murder, serial killer, gym, ...","[Marcia Karr, David James Campbell, Fritz Matt...","Two years ago, a young woman named Valerie was..."
The Russian Ninja,tt0100531,The Russian Ninja,NaN,NaN,87.0,1.798,[Swedish Action Film Force],"[kidnapping, ninja]","[Frederick Offrein, Helle Michaelsen, Mats Hud...",An ex-mercenary is enlisted back into the life...
Stone Cold,tt0431420,Stone Cold,NaN,NaN,87.0,13.679,"[Brandman Productions, Sony Pictures Televisio...","[cop, new england]","[Tom Selleck, Jane Adams, Viola Davis, Shawn R...",Jesse Stone is a former L.A. homicide detectiv...


Κάθε ταινία έχει πολλαπλές τιμές σε κάθε πεδίο. Για την επεξεργασία όλων των πληροφοριών θα πρέπει ή να δημιουργήσουμε μία βάση sql με πολλαπλούς πίνακες ή να δημιουργήσουμε διαφορετικά αρχεία csv που θα μπορούν να συνδυαστούν μεσω του κοινού πεδίου `imdb_id`. Παρακάτω παρουσιάζεται η δεύτερη μέθοδος. Τα αρχεία θα συνδυαστούν αργότερα στο πρόγραμμα Tableau μέσω των data relationships.

In [11]:
# Αρχείο companies.csv που θα περιέχει τις πληροφορίες των εταιριών παραγωγής
if not path.exists(path.join('datasets','companies.csv')):
    companies = (movies_tmdb[['imdb_id', 'production_companies']]
                 .reset_index()
                 .rename(columns={'index':'title'})
                 .explode(column='production_companies')
                 .dropna().reset_index(drop=True)).copy()
    companies.to_csv(path.join('datasets','companies.csv'), index=False)

In [12]:
# Αρχείο cast.csv που θα περιέχει τους ηθοποιούς ανά ταινία
if not path.exists(path.join('datasets','cast.csv')):
    cast = (movies_tmdb[['imdb_id', 'cast']]
             .reset_index()
             .rename(columns={'index':'title'})
             .explode(column='cast')
             .dropna().reset_index(drop=True)).copy()
    cast.to_csv(path.join('datasets','cast.csv'), index=False)

In [13]:
# Αρχείο keywords.csv που θα περιέχει τις λέξεις κλειδιά για την περιγραφή κάθε ταινίας
if not path.exists(path.join('datasets','keywords.csv')):
    keywords = (movies_tmdb[['imdb_id', 'keywords']]
                .reset_index().rename(columns={'index':'title'})
                .explode(column='keywords')
                .dropna().reset_index(drop=True).copy())
    keywords.to_csv(path.join('datasets', 'keywords.csv'), index=False)

In [14]:
# Οι υπόλοιπες στήλες θα αποθηκευτούν σε ένα αρχείο με όνομα movies_tmdb.csv
print("Στήλες: ", movies_tmdb.columns.values)

Στήλες:  ['imdb_id' 'original_title' 'budget' 'revenue' 'runtime' 'popularity'
 'production_companies' 'keywords' 'cast' 'overview']


In [15]:
if not path.exists(path.join('datasets','movies_tmdb.csv')):
    movies_tmdb.drop(columns=['production_companies', 'keywords', 'cast'], inplace=True)
    movies_tmdb.to_csv(path.join('datasets','movies_tmdb.csv'))

## Εξαγωγή πληροφοριών από την ιστοσελίδα Open Movie Database

Αρχικά προσπάθησα να χρησιμοποιήσω την βιβλιοθήκη `omdb` αλλά τελικά είχα καλύτερα αποτελέσματα με το api της ιστοσελίδας [OMDb](https://www.omdbapi.com). Η OMDb παρέχει τις ίδιες πληροφορίες που έχει το IMDB αλλά με ευκολότερη πρόσβαση. Οι πληροφορίες θα αποθηκευτούν στο λεξικό `omdb_info` και μετέπειτα σε αρχείο `movies_omdb.csv` με την βοήθεια του `pandas`.

In [16]:
if path.exists(path.join('datasets','movies_omdb.csv')):
    movies_omdb = pd.read_csv(path.join('datasets','movies_omdb.csv'), index_col=0)
else:
    from config import config # αποθήκευση των κλειδιών api στο βοηθητικό αρχείο 'config.py'
    omdb_info = {}
    for imdb_id in movies_tmdb['imdb_id'].to_list():
        try:
            res = requests.get(f"http://www.omdbapi.com/?i={imdb_id}&apikey={config['omdb_api_key']}", timeout=3).json()
            omdb_info[imdb_id] = [
                res['Year'],
                res['Rated'],
                res['Genre'],
                res['Director'],
                res['Writer'],
                res['Language'],
                res['Country'],
                res['Awards'],
                res['Metascore'],
                res['imdbRating'],
                res['imdbVotes'],
                [rating['Value'] for rating in res['Ratings'] if rating['Source'] == 'Rotten Tomatoes']
            ]
        except Exception:
            print(f"Request for movie id {imdb_id} did not executed.")

In [17]:
if not path.exists(path.join('datasets','movies_omdb.csv')):
    columns = ['year',
               'rated',
               'genre',
               'director',
               'writer',
               'language',
               'country',
               'awards',
               'metascore',
               'imdb_rating',
               'imdb_votes',
               'rotten_rating']
    movies_omdb = pd.DataFrame.from_dict(omdb_info, orient='index', columns=columns)
    movies_omdb = movies_omdb.reset_index().rename(columns={'index': 'imdb_id'})
    movies_omdb.replace({'N/A': np.nan}, inplace=True)
    
    # Βελτίωση των αποτελεσμάτων των ratings
    movies_omdb['imdb_votes'] = movies_omdb['imdb_votes'].str.replace(',', '').astype(int)
    movies_omdb['rotten_rating'] = movies_omdb.rotten_rating.apply(lambda x: x[0] if x else np.nan).str.replace('%', '').astype(float)
    movies_omdb['imdb_rating'] = movies_omdb.imdb_rating.astype(float)
    movies_omdb['metascore'] = movies_omdb.metascore.astype(float)

In [18]:
movies_omdb.sample(5)

,imdb_id,year,rated,genre,director,writer,language,country,awards,metascore,imdb_rating,imdb_votes,rotten_rating
31,tt0072666,1971,PG,"Horror, Sci-Fi","Don Barton, Arnold Stevens","Ron Kivett, Lee O. Larew, Don Barton",English,United States,NaN,NaN,2.1,5150,NaN
104,tt0086434,1983,R,"Action, Crime, Drama",Fabrizio De Angelis,"Fabrizio De Angelis, Dardano Sacchetti","Italian, English",Italy,NaN,NaN,5.2,532,NaN
103,tt0089695,1985,PG,"Action, Comedy, Crime",Corey Yuen,"See-Yuen Ng, Corey Yuen, Keith W. Strandberg",English,"United States, Hong Kong",NaN,30.0,5.6,17361,80.0
33,tt0085811,1983,PG,"Action, Adventure, Fantasy",Peter Yates,Stanford Sherman,English,"United Kingdom, United States",1 win & 5 nominations,45.0,6.1,33773,NaN
81,tt0117433,1997,R,"Action, Thriller",Sidney J. Furie,"Sidney J. Furie, Greg Mellott",English,"United States, Canada",NaN,NaN,4.5,579,NaN


In [19]:
movies_omdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   imdb_id        114 non-null    object 
 1   year           114 non-null    object 
 2   rated          104 non-null    object 
 3   genre          114 non-null    object 
 4   director       114 non-null    object 
 5   writer         113 non-null    object 
 6   language       114 non-null    object 
 7   country        114 non-null    object 
 8   awards         25 non-null     object 
 9   metascore      27 non-null     float64
 10  imdb_rating    114 non-null    float64
 11  imdb_votes     114 non-null    int64  
 12  rotten_rating  46 non-null     float64
dtypes: float64(3), int64(1), object(9)
memory usage: 11.7+ KB


Όπως και στο TMDB υπάρχουν πολλαπλές τιμές για κάθε πεδίο της εκάστοτε ταινίας οπότε δημιούργησα μία εφαρμογή που θα αποθηκεύει αυτές τις τιμές σε ξεχωριστό αρχείο csv μαζί με ένα αναγνωριστικό για την κάθε ταινία.

In [20]:
def create_sup_tables(column, f_name):
    """
    The fuction picks a dataframe column, splits values by coma,
    creates different columns and writes the new dataframe
    to a csv file based on f_name
    """
    df = movies_omdb[['imdb_id', column]].copy()
    df[column] = df[column].str.split(',')
    df = df.explode(column=column)
    df[column] = df[column].str.strip()
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    df.to_csv(path.join('datasets', f_name), index=False)

In [21]:
# Χρήση της εφαρμογής σε 5 στήλες

if not path.exists(path.join('datasets','genres.csv')):
    create_sup_tables('genre', 'genres.csv')
    
if not path.exists(path.join('datasets','writers.csv')):
    create_sup_tables('writer', 'writers.csv')
    
if not path.exists(path.join('datasets','countries.csv')):
    create_sup_tables('country', 'countries.csv')
    
if not path.exists(path.join('datasets','languages.csv')):
    create_sup_tables('language', 'languages.csv')
    
if not path.exists(path.join('datasets','directors.csv')):
    create_sup_tables('director', 'directors.csv')

Εξάγουμε τις υπόλοιπες στήλες για τις οποίες δεν δημιουργήσαμε ξεχωριστά αρχεία στο αρχείο `movies_omdb.csv`.

In [22]:
if not path.exists(path.join('datasets','movies_omdb.csv')):
    movies_omdb.drop(columns=['genre', 'writer', 'country', 'director', 'language'], inplace=True)
    movies_omdb.to_csv(path.join('datasets','movies_omdb.csv'))

Τέλος ενώνουμε τις πληροφορίες απο τις δύο ιστοσελίδες σε ένα αρχείο που ονομάζεται `movies.csv`.

In [23]:
movies = movies_tmdb.reset_index().rename(columns={'index':'title'}).merge(movies_omdb, how='inner', on='imdb_id')

movies.info()

if not path.exists(path.join('datasets','movies.csv')):
    movies.to_csv(path.join('datasets','movies.csv'), index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114 entries, 0 to 113
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           114 non-null    object 
 1   imdb_id         114 non-null    object 
 2   original_title  114 non-null    object 
 3   budget          35 non-null     float64
 4   revenue         25 non-null     float64
 5   runtime         113 non-null    float64
 6   popularity      114 non-null    float64
 7   overview        114 non-null    object 
 8   year            114 non-null    object 
 9   rated           104 non-null    object 
 10  awards          25 non-null     object 
 11  metascore       27 non-null     float64
 12  imdb_rating     114 non-null    float64
 13  imdb_votes      114 non-null    int64  
 14  rotten_rating   46 non-null     float64
dtypes: float64(7), int64(1), object(7)
memory usage: 14.2+ KB


Στο πρόγραμμα Tableau χρησιμοποίησα το αρχείο `movies.csv` ως το κύριο αρχείο και το συνδύασα με τα αρχεία:

- `cast.csv`
- `companies.csv`
- `countries.csv`
- `directors.csv`
- `genres.csv`
- `keywords.csv`
- `languages.csv`
- `writers.csv`

για επιπλεον πληροφορίες.